In [104]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/images-for-testing/Huanglongbing.jpg
/kaggle/input/images-for-testing/pepper_bell.jpg


In [105]:
! pip install tensorflow-datasets

In [106]:
!pip install protobuf==3.20.3 --quiet

In [109]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf

tf.__version__

# loading the crop-image data
dataset, info = tfds.load('plant_village', split=['train[:80%]', 'train[80%:]'], as_supervised=True, with_info=True)
print(dataset)

train, test = dataset

[<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, <_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>]


In [ ]:
def pre_process(image, label):
    image = tf.image.resize(image, (128,128)) / 255.0
    return image, label

batch_size = 32

train_dataset = train.map(pre_process).batch(batch_size).shuffle(1000)
test_dataset = test.map(pre_process).batch(batch_size)

In [ ]:
! pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

# Get a single batch of images and labels
image, label = train.as_numpy_iterator().next()

# Create a figure
fig, axes = plt.subplots(1, 1)  # One subplot for a single image

# Display the image
axes.imshow(image)  
axes.set_title(f"Label: {label}")
axes.axis("off")  # Hide axes

plt.show()


In [ ]:
classes = info.features['label'].num_classes
names = info.features['label'].names
names, classes

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(128,128,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(512,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(512,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(classes, activation='softmax')
])

model.summary()

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_dataset, validation_data=test_dataset, epochs=20)

In [113]:
import kagglehub
import cv2

images_for_testing_path = kagglehub.dataset_download('rameshkuntigorla/images-for-testing')

image = cv2.imread(f'{images_for_testing_path}/tomato_early_blight.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.resize(image,(128,128))
image = image.astype(np.float32)/255.0
image

array([[[0.40784314, 0.35686275, 0.3529412 ],
        [0.41960785, 0.3647059 , 0.36078432],
        [0.40392157, 0.34901962, 0.34901962],
        ...,
        [0.7490196 , 0.6666667 , 0.56078434],
        [0.69411767, 0.6039216 , 0.5411765 ],
        [0.7254902 , 0.63529414, 0.57254905]],

       [[0.43137255, 0.3764706 , 0.37254903],
        [0.4117647 , 0.35686275, 0.3529412 ],
        [0.44313726, 0.3882353 , 0.38431373],
        ...,
        [0.7137255 , 0.6313726 , 0.5529412 ],
        [0.7058824 , 0.61960787, 0.5647059 ],
        [0.6039216 , 0.5176471 , 0.4627451 ]],

       [[0.45882353, 0.40392157, 0.4       ],
        [0.44313726, 0.3882353 , 0.38431373],
        [0.46666667, 0.4117647 , 0.40784314],
        ...,
        [0.72156864, 0.6509804 , 0.5647059 ],
        [0.63529414, 0.5647059 , 0.5176471 ],
        [0.46666667, 0.39607844, 0.34901962]],

       ...,

       [[0.38039216, 0.32156864, 0.3019608 ],
        [0.5294118 , 0.45490196, 0.43137255],
        [0.80784315, 0

In [115]:
predictions = model.predict(np.expand_dims(image, axis=0))
class_index = np.argmax(predictions)
names[class_index]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


'Tomato___Late_blight'

In [116]:
model.save('crop_pred.keras')